In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import joblib
import os
import csv
import re
import warnings
warnings.filterwarnings('ignore')


In [72]:
def train_XGBmodel(motif,data):
    motif_data = data.loc[data['kmer']==motif,:]

    X_train, X_test, y_train, y_test = train_test_split(motif_data.iloc[:,2:motif_data.shape[1]-1],
                                                        motif_data['label'],
                                                        train_size=0.9, test_size=0.1)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    xgboost = XGBClassifier()
    xgboost.fit(X_train, y_train)
    auc = roc_auc_score(y_test, xgboost.predict(X_test))

    return(auc,xgboost,scaler)

In [73]:
data = pd.read_csv('/home/jiayi/5moU/data/DL_fromTombo/Tombo_all_avg.csv')
data

,indx,kmer,avg_mean,avg_std,avg_mdintense,avg_length2,label
0,20,CATCT,-0.379295,0.059457,-0.386073,18.6,1
1,22,TCTTC,-0.224518,0.079366,-0.226347,21.8,1
2,23,CTTCG,-0.027033,0.106017,-0.016552,24.2,1
3,35,CGTGG,0.135500,0.146305,0.127863,19.6,1
4,44,CCTGC,-0.297275,0.098792,-0.307865,59.0,1
...,...,...,...,...,...,...,...
1822819,350,GCTGG,0.251371,0.097341,0.250863,205.6,0
1822820,365,GATCC,-0.242580,0.199999,-0.230983,8.0,0
1822821,374,GATCC,-0.153419,0.253634,-0.074312,570.2,0
1822822,377,CCTGA,-0.008397,0.117552,-0.003787,137.6,0


In [74]:
df_group = data.groupby(['kmer','label']).agg(coverage = ('indx','count'))
df_group

coverage
kmer  label          
ACTAC 0         26917
      1          3118
ACTGG 0         12301
      1          1549
ACTTC 0         18476
...               ...
TGTAC 1          1022
TGTGC 0         12227
      1          1612
TGTTC 0         39714
      1          4622

[108 rows x 1 columns]

In [75]:
df_group = pd.read_csv('/home/jiayi/5moU/Code/DL5mou/DeepRead/motif_count.csv')
df_group=df_group[['kmer','label','coverage']]

In [76]:
#df = data.groupby(['kmer','label']).agg(coverage = ('indx','count'))
df_group.to_csv('/home/jiayi/5moU/Code/DL5mou/DeepRead/motif_count.csv',index=False)
df_group = pd.read_csv('/home/jiayi/5moU/Code/DL5mou/DeepRead/motif_count.csv')
df_group

,kmer,label,coverage
0,ACTAC,0,26917
1,ACTAC,1,3118
2,ACTGG,0,12301
3,ACTGG,1,1549
4,ACTTC,0,18476
...,...,...,...
103,TGTAC,1,1022
104,TGTGC,0,12227
105,TGTGC,1,1612
106,TGTTC,0,39714


In [77]:
df1 = df_group.loc[df_group['label']==1,:]
motifs1 = df1.loc[df1['coverage'] > 50,'kmer']
df0 = df_group.loc[df_group['label']==0,:]
motifs2 = df0.loc[df0['coverage'] > 50,'kmer']

In [78]:
motifs = np.intersect1d(motifs1,motifs2)

In [79]:
file = open('/home/jiayi/5moU/data/DL_fromTombo/perf.csv','w',encoding = 'UTF-8')

In [80]:
csv_writer = csv.writer(file)

In [81]:
csv_writer.writerow(['motif','auc'])

11

In [82]:
for motif in motifs:
    try:
        auc,model,scaler = train_XGBmodel(motif,data)
        joblib.dump(model, '/home/jiayi/5moU/Code/DL5mou/DeepRead/model/' + motif + '.joblib')
        joblib.dump(scaler,'/home/jiayi/5moU/Code/DL5mou/DeepRead/scaler/' + motif + '.joblib')
        csv_writer.writerow([motif,auc])
    except Exception as e:
        print(str(e))
        continue

file.close()


# predict

In [52]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import joblib
import os
import csv
import re
import warnings
warnings.filterwarnings('ignore')

In [98]:
df_perf = pd.read_csv('/home/jiayi/5moU/data/DL_fromTombo/perf.csv')
motifs = df_perf['motif']


test = pd.read_csv('/home/jiayi/5moU/data/DL_fromTombo/Tombo_all_avg.csv')
test = test.iloc[:,1:(test.shape[1])]
test



,kmer,avg_mean,avg_std,avg_mdintense,avg_length2,label
0,CATCT,-0.379295,0.059457,-0.386073,18.6,1
1,TCTTC,-0.224518,0.079366,-0.226347,21.8,1
2,CTTCG,-0.027033,0.106017,-0.016552,24.2,1
3,CGTGG,0.135500,0.146305,0.127863,19.6,1
4,CCTGC,-0.297275,0.098792,-0.307865,59.0,1
...,...,...,...,...,...,...
1822819,GCTGG,0.251371,0.097341,0.250863,205.6,0
1822820,GATCC,-0.242580,0.199999,-0.230983,8.0,0
1822821,GATCC,-0.153419,0.253634,-0.074312,570.2,0
1822822,CCTGA,-0.008397,0.117552,-0.003787,137.6,0


In [97]:
for motif in motifs:
    try:
        model = joblib.load('/home/jiayi/5moU/Code/DL5mou/DeepRead/model/' + motif + '.joblib')
        scaler = joblib.load('/home/jiayi/5moU/Code/DL5mou/DeepRead/scaler/' + motif + '.joblib')
        feature = scaler.transform(test.loc[test['kmer'] == motif].iloc[:,1:test.shape[1]-1]
        test.loc[test['kmer'] == motif, 'prob'] = model.predict_proba(feature)[:, 1]
        test.loc[test['kmer'] == motif, 'prediction'] = model.predict(feature)
                               
    except Exception as e:
    print(str(e))
    continue

SyntaxError: invalid syntax (1131984711.py, line 9)

In [ ]:
test = test.loc[:,['chrom','chrom_pos','strand','read_id','read_pos','motif','prediction','prob']]
test.to_csv('/data/yuxin/vaccine_QC/M_pred.csv',index = 0)